In [1]:
import os

In [2]:
%pwd


'/home2/cslab/Music/Mtech-projects/Deploying-Customer-Churn-model/notebook'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home2/cslab/Music/Mtech-projects/Deploying-Customer-Churn-model'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    num_leaves: int
    learning_rate: float
    max_depth: int
    target_column: str

In [6]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LightGBM
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            num_leaves=params.num_leaves,
            learning_rate=params.learning_rate,
            max_depth=params.max_depth,
            target_column=schema.name
        )

        return model_trainer_config


In [8]:
import pandas as pd
import os
from mlFlowProject import logger
from lightgbm import LGBMClassifier
import joblib


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        print(train_data.info())
        print(test_data.shape)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lgbm_clf = LGBMClassifier(n_estimators=self.config.n_estimators, num_leaves=self.config.num_leaves, learning_rate=self.config.learning_rate, max_depth= self.config.max_depth)
        lgbm_clf.fit(train_x, train_y)

        joblib.dump(lgbm_clf, os.path.join(self.config.root_dir, self.config.model_name))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-02-06 09:52:12,271: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-06 09:52:12,273: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-06 09:52:12,274: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-06 09:52:12,275: INFO: common: created directory at: artifacts]
[2025-02-06 09:52:12,275: INFO: common: created directory at: artifacts/model_trainer]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7498 entries, 0 to 7497
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   num_pipeline__RowNumber          7498 non-null   float64
 1   num_pipeline__CustomerId         7498 non-null   float64
 2   num_pipeline__CreditScore        7498 non-null   float64
 3   num_pipeline__Gender             7498 non-null   float64
 4   num_pipeline__Age                7498 non-null   float64
 5   num_pipeline__Balance            74

/home2/cslab/miniconda3/envs/cancer_detection/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home2/cslab/miniconda3/envs/cancer_detection/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
